# 事前准备

In [1]:
import pandas as pd
import numpy as np
import os, time, math
import collections

OUTPUT_FILE = 'CISS_20240612'

In [2]:
cur_dir = os.getcwd()
relative_data_dir = os.path.join(cur_dir,
                                 'raw_data', 'NASS_CISS_data', 'CISS_raw')

In [3]:
relative_data_dir

'F:\\Code_reposity\\PyProjects\\nasscissInjuryAnalyze\\nasscissInjuryAnalysis\\src\\CreateData\\raw_data\\NASS_CISS_data\\CISS_raw'

# 数据读入并删除重复列

In [4]:
wanted_table_list = ['CRASH', 'EVENT', 'CDC', 'GV', 'OCC']

raw_data = dict()

for table in wanted_table_list:
    print('Extracting table: %s ...\n' % table)
    raw_data[table] = pd.DataFrame()
    for year in range(2017, 2023):
        tmp = pd.read_sas(
            os.path.join(relative_data_dir, str(year), '%s.sas7bdat' % table))
        raw_data[table] = pd.concat([raw_data[table], tmp])
        raw_data[table].reset_index(inplace=True, drop=True)
    raw_data[table] = raw_data[table].dropna(how='all', axis=1)

merge_centre = 'OCC'
len(raw_data[merge_centre])


Extracting table: CRASH ...

Extracting table: EVENT ...

Extracting table: CDC ...

Extracting table: GV ...

Extracting table: OCC ...



34873

In [5]:
for table in wanted_table_list:
    raw_data[table] = raw_data[table].drop(
        columns=['CASEID', 'PSU', 'CASENO', 'CATEGORY'])
    print(table, 'with cases counted: %s' % len(raw_data[table]))

    if table != 'CRASH':
        raw_data[table] = raw_data[table].drop(
            columns=['CASEWGT', 'PSUSTRAT', 'VERSION'])

raw_data['CDC'] = raw_data['CDC'].drop(columns=['DVTOTAL'])


CRASH with cases counted: 17459
EVENT with cases counted: 32832
CDC with cases counted: 36936
GV with cases counted: 31084
OCC with cases counted: 34873


# 以OCC为中心进行结合

In [6]:
merge_key = dict()
for table in wanted_table_list:
    if table == 'CRASH':
        merge_key[table] = ['CASENUMBER']
    if table in ['GV', 'CDC']:
        merge_key[table] = ['CASENUMBER', 'VEHNO']
    if table == 'OCC':
        merge_key[table] = ['CASENUMBER', 'VEHNO', 'OCCNO']
        
ego_data = raw_data[merge_centre].copy(deep=True).drop(columns=['RACE'])
other_data = raw_data['GV'].copy(deep=True)

for table in ['GV']:
    ego_data = pd.merge(ego_data, raw_data[table], on=merge_key[table], how='inner')
for table in ['CRASH', 'CDC']:
    ego_data = pd.merge(ego_data, raw_data[table], on=merge_key[table], how='left')
    other_data = pd.merge(other_data, raw_data[table], on=merge_key[table], how='left')
    
for data in [ego_data, other_data]:
    data.reset_index(drop=True, inplace=True)


In [7]:
raw_data['OCC']['WEIGHT'].value_counts()

WEIGHT
999.0    14386
82.0       986
68.0       930
73.0       915
91.0       886
         ...  
228.0        1
190.0        1
189.0        1
217.0        1
213.0        1
Name: count, Length: 196, dtype: int64

# 处理特征

## 特征名统一化

In [8]:
factor_list_ciss = {
    'vehicle': {
        'MODELYR': 'Model Year',
        'CURBWT': 'Curb Weight',  # 要用吗？？？
        #         'TRAVELSP': 'Travel Speed',
        'PREMOVE': 'premovement before collision',  # -- 还需要检查如何分类！
        'MANEUVER': 'Maneuver before collision',  # -- 还需要检查如何分类！
        'PDOF': 'Clock-form Direction of force',
        'BODYTYPE': 'Body Type',
        #         'otbdytyp': 'Body Type of the other vehicle'  # 对方车辆情报
    },
    'driver': {
        'DRPRESENT': 'Driver Present',  # 用于筛选
        'PARALCOHOL': 'Alcohol Present',
        # 'DRUGS': 'Drug Present',
        'DISTRACT':
        'Distracted in Driving',  # 注意CISS中DISTRACT=9为未知（但是一半都是缺损也太多了！）
        'SEX': 'Sex',  # 3~6 为怀孕 （时间越长编号越大）
        'AGE': 'Age',
        'HEIGHT': 'height',
        'WEIGHT': 'weight',
        'RACE': 'Race'  # 0612追加    ---  更新：欠缺值太多了，怎么办哇！
    },
    'environment': {
        'SPEEDLIMIT': 'Speed Limit',  # 0为无速度限制 -- 换成大值！-- 还需要检查如何分类！
        'RELTOJUNCT': 'Related to Intersection',
        'TRAFFLOW': 'Traffic Flow Situation',
        'RDLANES': 'Number of lanes',
        'ALIGNMENT': 'Alignment of Road',
        'PROFILE': 'Uphill or Downhill',  # 用吗？
        'SURFTYPE': 'Surface Type',  # 用于筛选-- 还需要检查如何分类！
        'SURFCOND': 'Surface Condition',  # 路面条件，用吗？-- 还需要检查如何分类！
        'LIGHTCOND': 'Lighting Condition',
        'WEATHER': 'Climate',  # -- 还需要检查如何分类！
        'TRAFDEV': 'Traffic Condition',  # 是否有学校等减速，用吗？  -- 还需要检查如何分类！
        # 'PREEVENT': 'Pre-crash Event', # 事故前事件，用吗？  -- 还需要检查如何分类！
        'PRELOC': 'Pre-event Location',
        'TRAFFUNCT': 'Traffic Conrtol Functioning',
        'MANCOLL': 'Crash Type'  # 引用硕士研究说明这个东西可以被预测！  -  这个已经帮忙分好类了！！！
    },
    'time series': {
        'CRASHMONTH': 'month',
        'CRASHYEAR': 'year',
        'DAYOFWEEK': 'Day in Week'
    },
    'post crash': {
        'DVTOTAL': 'delta v',  # 0612追加 （for compare）
        'DVCONF': 'delta v confidence level'
    }
}

In [9]:
included = 1
feature_count = 0

for cate, features in factor_list_ciss.items():
    for factor in features.keys():
        feature_count += 1
        if factor not in ego_data.columns:
            print('No existance! %s' %factor)
            included = 0

if included:
    print('No factor missed!')
print('Feature number is: %s' %feature_count)

No factor missed!
Feature number is: 33


## 处理Nan值 & 数据筛选（初筛：将所有特征的未定义转换成nan  - 客观）

- 注：并未dropna所以不影响！真正dropna在第三次筛选）


In [10]:
deal_ego = ego_data.copy(deep=True)
# deal_other = other_data.copy(deep=True)

In [11]:
nanlist_ciss = [
    # vehicle
    ['MODELYR', 9999],
    ['CURBWT', 9999],
    ['PREMOVE', 99],
    ['PREMOVE', 98],
    ['MANEUVER', 98],
    ['MANEUVER', 99],
    ['BODYTYPE', 99],
    ['BODYTYPE', 98],
    ['PDOF', 998],
    ['PDOF', 999],

    # driver
    ['DRPRESENT', 9],
    ['PARALCOHOL', 7],
    ['PARALCOHOL', 8],
    ['PARALCOHOL', 9],
    ['DISTRACT', 9],
    ['SEX', 9],
    ['HEIGHT', 999],
    ['WEIGHT', 999],
    ['AGE', 999],
    ['RACE', 7],
    ['RACE', 8],
    ['RACE', 9],

    # environment
    ['SPEEDLIMIT', 999],
    ['RELTOJUNCT', 9],
    ['TRAFFLOW', 9],
    ['RDLANES', 9],
    ['ALIGNMENT', 9],
    ['PROFILE', 9],
    ['SURFTYPE', 8],
    ['SURFTYPE', 9],
    ['SURFCOND', 98],
    ['SURFCOND', 99],
    ['LIGHTCOND', 9],
    ['WEATHER', 98],
    ['WEATHER', 99],
    ['TRAFDEV', 9],
    ['PRELOC', 9],
    ['MANCOLL', 0],
    ['TRAFFUNCT', 9],
    ['MANCOLL', 9],

    # OBJECTIVE
    ['MAIS', 9],
    ['MAIS', 99],
    
    # Compare
    ['DVTOTAL', 999]
]

# for i, data in enumerate([deal_ego, deal_other]):
for i, data in enumerate([deal_ego]):
    for target in nanlist_ciss:
        if target[0] in data:
            initial_nan = data[target[0]].isnull().sum()
            data.loc[data[target[0]] == target[1], target[0]] = np.nan
            processed = data[target[0]].isnull().sum()
            print(
                "The table %s got null at feature %s before processing: %s and after processing it got: %s more"
                % (i + 1, target[0], initial_nan, processed - initial_nan))
            if processed - initial_nan > len(data) / 10:
                print("*" * 50, "\n Over 10% nan of feature: ", target[0])
                print("*" * 50)

The table 1 got null at feature MODELYR before processing: 0 and after processing it got: 2 more
The table 1 got null at feature CURBWT before processing: 0 and after processing it got: 289 more
The table 1 got null at feature PREMOVE before processing: 0 and after processing it got: 46 more
The table 1 got null at feature PREMOVE before processing: 46 and after processing it got: 37 more
The table 1 got null at feature MANEUVER before processing: 0 and after processing it got: 816 more
The table 1 got null at feature MANEUVER before processing: 816 and after processing it got: 19643 more
************************************************** 
 Over 10% nan of feature:  MANEUVER
**************************************************
The table 1 got null at feature BODYTYPE before processing: 0 and after processing it got: 0 more
The table 1 got null at feature BODYTYPE before processing: 0 and after processing it got: 0 more
The table 1 got null at feature PDOF before processing: 4600 and afte

In [12]:
# for data in [deal_ego, deal_other]:
for data in [deal_ego]:
    for category in factor_list_ciss.keys():
        for old_name, new_name in factor_list_ciss.get(category).items():
            if old_name in data.columns:
                data[new_name] = data[old_name]

## 分车种

In [14]:
# for data in [deal_ego, deal_other]:    
for data in [deal_ego]:
    for prev_col, new_col in zip(['BODYTYPE'], ['Body Category']):
        data.loc[(data[prev_col] <= 9) | (data[prev_col] == 11) |
                 (data[prev_col] == 12) | (data[prev_col] == 17),
                 new_col] = 0  #Sedan
        data.loc[((data[prev_col] >= 14) & (data[prev_col] <= 16)) |
                 (data[prev_col] == 19), new_col] = 1  #SUV
        data.loc[((data[prev_col] >= 20) & (data[prev_col] <= 29)) |
                 (data[prev_col] == 60), new_col] = 2  #Van
        data.loc[(data[prev_col] == 10) | ((data[prev_col] >= 30) &
                                           (data[prev_col] <= 39)) |
                 (data[prev_col] == 67), new_col] = 3  #Pickup
        

## 对于可预见事故，获取对方信息

In [15]:
# other_col_list = ['Clock-form Direction of force', 'Body Category', 'premovement before collision']

# df_other = deal_other.copy(deep=True).drop_duplicates(subset=['CASENUMBER', 'VEHNO', 'EVENTNO'] + other_col_list)
# df_other['VEHNO'] = df_other['OBJCONT']
# new_col_list = []

# for col in other_col_list:
#     ocol = 'Other Veh ' + col
#     df_other[ocol] = df_other[col]
#     new_col_list.append(ocol)
# df_other = df_other[new_col_list + ['CASENUMBER', 'VEHNO', 'EVENTNO']]
# data_with_oinfo = pd.merge(deal_ego, df_other, on=['CASENUMBER', 'VEHNO', 'EVENTNO'], how='left')
# data_with_oinfo = data_with_oinfo.dropna(subset=['MAIS'])


In [ ]:
# tmp = data_with_oinfo
# for col in new_col_list:
#     print("Feature %s ,Percentage of data restored: %s" % (col, round(1 - len(tmp[col].dropna()) / len(tmp), 3)))

## 进一次数据筛选（法律，车辆数，车辆类型） - 客观

In [17]:
query_conditions = [
    'VEHICLES == 2',  # 两车相撞
    'ROLE == 1',  # 指定驾驶员
    'BODYTYPE < 80 & BODYTYPE != 13',  # 车辆类型限定
    'PREMOVE != 0',  # 有驾驶员
    'MANEUVER != 0',  # 同上
    'DISTRACT != 0',  # 同上
    'AGE >= 15',  # 法律上
    'PRELOC != 0',
    'DVRANK == 1',  # 最大DV
    'MAIS < 7'
]

# data_with_selection = data_with_oinfo.copy(deep=True)
data_with_selection = deal_ego.copy(deep=True)

for que in query_conditions:
    prev_len = len(data_with_selection)
    data_with_selection = data_with_selection.query(que)
    print('%s: cases: %s before -> %s left. True ratio: %s' %
          (que, prev_len, len(data_with_selection),
           int(len(data_with_selection.query('MAIS > 2')) / len(data_with_selection)*10000) / 10000))
data_with_selection.reset_index(drop=True, inplace=True)

VEHICLES == 2: cases: 55245 before -> 28699 left. True ratio: 0.0639
ROLE == 1: cases: 28699 before -> 19549 left. True ratio: 0.0657
BODYTYPE < 80 & BODYTYPE != 13: cases: 19549 before -> 19549 left. True ratio: 0.0657
PREMOVE != 0: cases: 19549 before -> 19549 left. True ratio: 0.0657
MANEUVER != 0: cases: 19549 before -> 19549 left. True ratio: 0.0657
DISTRACT != 0: cases: 19549 before -> 19545 left. True ratio: 0.0657
AGE >= 15: cases: 19545 before -> 19358 left. True ratio: 0.0663
PRELOC != 0: cases: 19358 before -> 19358 left. True ratio: 0.0663
DVRANK == 1: cases: 19358 before -> 12482 left. True ratio: 0.0572
MAIS < 7: cases: 12482 before -> 10488 left. True ratio: 0.068


In [ ]:
# for oinfo in new_col_list:
#     print('Info name: %s, Valid oinfo case: %s, with ratio: %s' %(
#         oinfo, 
#         sum(data_with_selection[oinfo].value_counts()), 
#         sum(data_with_selection[oinfo].value_counts()) / len(data_with_selection))
#         )

## 提取需要的列

In [18]:
# wanted_features = ['MAIS', 'Body Category'] + new_col_list
wanted_features = ['MAIS', 'Body Category']

for cate in factor_list_ciss.keys():
    for sas_name, feature_name in factor_list_ciss.get(cate).items():
        if sas_name in ('BODYTYPE', 'DRPRESENT'):
            continue
        wanted_features.append(feature_name)
        
wanted_features, len(wanted_features)


(['MAIS',
  'Body Category',
  'Model Year',
  'Curb Weight',
  'premovement before collision',
  'Maneuver before collision',
  'Clock-form Direction of force',
  'Alcohol Present',
  'Distracted in Driving',
  'Sex',
  'Age',
  'height',
  'weight',
  'Race',
  'Speed Limit',
  'Related to Intersection',
  'Traffic Flow Situation',
  'Number of lanes',
  'Alignment of Road',
  'Uphill or Downhill',
  'Surface Type',
  'Surface Condition',
  'Lighting Condition',
  'Climate',
  'Traffic Condition',
  'Pre-event Location',
  'Traffic Conrtol Functioning',
  'Crash Type',
  'month',
  'year',
  'Day in Week',
  'delta v',
  'delta v confidence level'],
 33)

In [19]:
for info in wanted_features:
    true_info_ratio = sum(data_with_selection[info].value_counts()) / len(data_with_selection)
    if true_info_ratio < 0.8:
        print('Info name: %s, Valid info case: %s, with ratio: %s' %(
            info, 
            sum(data_with_selection[info].value_counts()), 
            round(true_info_ratio, 3)
            ))
        if true_info_ratio < 0.5:
            print( "*" * 50, '\n', info, 'Warining: 数据少一半以上！\n', "*" * 50)

Info name: Maneuver before collision, Valid info case: 7512, with ratio: 0.716
Info name: Distracted in Driving, Valid info case: 6822, with ratio: 0.65
Info name: height, Valid info case: 7239, with ratio: 0.69
Info name: weight, Valid info case: 7529, with ratio: 0.718
Info name: Race, Valid info case: 7175, with ratio: 0.684
Info name: delta v, Valid info case: 7430, with ratio: 0.708


## 三次筛选（考虑到使用特征的筛选） - 主观筛选
考虑到预测时候可能不需要Maneuver因为太接近事故发生，所以dropna时候就不drop它！

dropna时不drop的属性： MANEUVER, OPDOF, OTRAVELSP, OPREMOVE, PARALCOHOL, DISTRACT

In [20]:
no_dropna_cols = [
    'Other Veh Clock-form Direction of force',
    'Other Veh premovement before collision',
    'Alcohol Present',
    'Distracted in Driving',
    'Maneuver before collision',
    'Other Veh Body Category',
    'delta v',
    'delta v confidence level',
    ]

dropna_cols = list(set(wanted_features) - set(no_dropna_cols))
dropna_cols

['Number of lanes',
 'Day in Week',
 'Body Category',
 'weight',
 'Related to Intersection',
 'Lighting Condition',
 'Race',
 'Alignment of Road',
 'month',
 'Traffic Flow Situation',
 'Climate',
 'MAIS',
 'Uphill or Downhill',
 'Pre-event Location',
 'Surface Condition',
 'Clock-form Direction of force',
 'Speed Limit',
 'Age',
 'premovement before collision',
 'Traffic Conrtol Functioning',
 'Sex',
 'year',
 'Model Year',
 'Crash Type',
 'height',
 'Traffic Condition',
 'Curb Weight',
 'Surface Type']

In [21]:
data_dropna = data_with_selection.copy(deep=True).dropna(subset=dropna_cols)
# data_dropna = data_dropna[wanted_features]
data_dropna.reset_index(drop=True, inplace=True)
data_dropna

,CASENUMBER,VEHNO,OCCNO,SEATLOC,AGE,HEIGHT,WEIGHT,SEX,FETALMORT,ROLE,...,Traffic Condition,Pre-event Location,Traffic Conrtol Functioning,Crash Type,month,year,Day in Week,delta v,delta v confidence level,Body Category
0,b'1-10-2017-003-09',1.0,1.0,11.0,30.0,170.0,93.0,4.0,0.0,1.0,...,0.0,1.0,0.0,5.0,1.0,2017.0,5.0,NaN,0.0,0.0
1,b'1-10-2017-016-03',1.0,1.0,11.0,75.0,157.0,72.0,2.0,8.0,1.0,...,0.0,1.0,0.0,4.0,3.0,2017.0,6.0,17.0,1.0,0.0
2,b'1-10-2017-017-03',1.0,1.0,11.0,19.0,170.0,79.0,2.0,8.0,1.0,...,1.0,1.0,2.0,4.0,3.0,2017.0,5.0,30.0,1.0,0.0
3,b'1-10-2017-017-03',2.0,1.0,11.0,47.0,163.0,104.0,2.0,8.0,1.0,...,1.0,1.0,2.0,4.0,3.0,2017.0,5.0,21.0,1.0,0.0
4,b'1-10-2017-020-03',2.0,1.0,11.0,59.0,157.0,113.0,2.0,8.0,1.0,...,1.0,1.0,2.0,4.0,4.0,2017.0,4.0,9.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5275,b'1-77-2022-009-03',2.0,1.0,11.0,24.0,178.0,77.0,2.0,8.0,1.0,...,0.0,1.0,0.0,4.0,2.0,2022.0,5.0,NaN,0.0,1.0
5276,b'1-77-2022-036-03',1.0,1.0,11.0,19.0,188.0,91.0,1.0,8.0,1.0,...,1.0,1.0,2.0,4.0,4.0,2022.0,4.0,15.0,4.0,1.0
5277,b'1-77-2022-086-04',1.0,1.0,11.0,24.0,157.0,50.0,2.0,8.0,1.0,...,1.0,1.0,2.0,4.0,10.0,2022.0,7.0,34.0,1.0,0.0
5278,b'1-77-2022-089-04',2.0,1.0,11.0,59.0,185.0,91.0,1.0,8.0,1.0,...,1.0,1.0,2.0,4.0,11.0,2022.0,6.0,14.0,4.0,1.0


In [22]:
for col in wanted_features:
    print(col, '___', round(sum(data_dropna[col].value_counts()) / len(data_dropna), 3))

MAIS ___ 1.0
Body Category ___ 1.0
Model Year ___ 1.0
Curb Weight ___ 1.0
premovement before collision ___ 1.0
Maneuver before collision ___ 0.849
Clock-form Direction of force ___ 1.0
Alcohol Present ___ 0.857
Distracted in Driving ___ 0.78
Sex ___ 1.0
Age ___ 1.0
height ___ 1.0
weight ___ 1.0
Race ___ 1.0
Speed Limit ___ 1.0
Related to Intersection ___ 1.0
Traffic Flow Situation ___ 1.0
Number of lanes ___ 1.0
Alignment of Road ___ 1.0
Uphill or Downhill ___ 1.0
Surface Type ___ 1.0
Surface Condition ___ 1.0
Lighting Condition ___ 1.0
Climate ___ 1.0
Traffic Condition ___ 1.0
Pre-event Location ___ 1.0
Traffic Conrtol Functioning ___ 1.0
Crash Type ___ 1.0
month ___ 1.0
year ___ 1.0
Day in Week ___ 1.0
delta v ___ 0.776
delta v confidence level ___ 1.0


# 补充说明

- 以下的筛选均基于第一轮筛选（即最开始筛去nan）
* 如果根本不使用maneuver的话是不是就没有筛它的必要啊？！ 
    * 0307:没筛！

* 是否喝酒也根据是否使用去进行筛选！ 
    * 0307：没筛！
    
    
- 0308: 对齐两边数据并且完成初步数据集作成

In [ ]:
nass_feas = set([
    'MAIS', 'Other Veh Body Category', 'Body Category',
    'Other Veh Clock-form Direction of force', 'Other Veh TRAVELSP',
    'Other Veh premovement before collision', 'Model Year', 'Curb Weight',
    'Travel Speed', 'premovement before collision',
    'Maneuver before collision', 'Clock-form Direction of force',
    'Alcohol Present', 'Distracted in Driving', 'Sex', 'Age', 'height',
    'weight', 'Speed Limit', 'Related to Intersection',
    'Traffic Flow Situation', 'Number of lanes', 'Alignment of Road',
    'Uphill or Downhill', 'Surface Type', 'Surface Condition',
    'Lighting Condition', 'Climate', 'Traffic Condition', 'Pre-event Location',
    'Traffic Conrtol Functioning', 'Crash Type', 'month', 'year', 'Day in Week'
])

ciss_feas = set(data_dropna.columns)

nass_feas - ciss_feas

# 保存数据

In [23]:
data_dropna.to_csv(os.path.join(os.path.dirname(os.curdir),
                                'CreatedData/CISS/',
                                OUTPUT_FILE + '_dropna.csv'),
                   encoding='utf-8')

In [24]:
data_with_selection.to_csv(os.path.join(os.path.dirname(
    os.curdir), 'CreatedData/CISS/', OUTPUT_FILE + '_NoDropAllFeatures.csv'),
                           encoding='utf-8')

In [25]:
data_dropna['CASEWGT']

0       1098.361402
1        207.980576
2        176.573624
3        176.573624
4        405.610229
           ...     
5275     321.810619
5276     644.040510
5277    4911.585968
5278     994.960746
5279    3380.364729
Name: CASEWGT, Length: 5280, dtype: float64